# Summarising compute speed

`benchmark.py` was run against the REFSOIL collection -- 1015 whole bacterial genomes. We separately recorded times to:
- load the sequences from compressed files and convert them into  `SeqRecord` objects
- identify the divergent set

Each condition was run 5 times with the sequences were randomly drawn without replacement.

# Synopsis

Performance is approximately linear with the number of sequences.

**TODO** actually plot the best fit line, not my little hack.

In [1]:
from collections import defaultdict
from cogent3 import load_table, make_table

import plotly.graph_objects as go

In [2]:
def summarise(table):
    columns = "k", "num_seqs"
    distinct = table.distinct_values(columns)
    data = defaultdict(list)
    for k, num_reps in distinct:
        subtable = table.filtered(lambda x: tuple(x) == (k, num_reps), columns=columns)
        time_s2r = subtable.columns["time(s2r)"]
        time_maxd = subtable.columns["time(maxd)"]
        data["k"].append(k)
        data["num_seqs"].append(num_reps)
        data["mean(s2r)"].append(time_s2r.mean())
        data["stdev(s2r)"].append(time_s2r.std(ddof=1))
        data["mean(maxd)"].append(time_maxd.mean())
        data["stdev(maxd)"].append(time_maxd.std(ddof=1))
    return make_table(data=data).sorted(columns=columns)


table = load_table("benchmark.tsv")
summary = summarise(table)
summary

3,50,21.7792,1.6411,0.0957,0.0123
3,100,44.5110,2.1210,0.1291,0.0411
3,150,61.9987,1.4671,0.1475,0.0452
3,200,80.0735,1.8596,0.1820,0.0344
5,50,20.5672,1.1128,0.4244,0.0722
5,100,43.1821,1.7683,0.6842,0.1611
5,150,64.8690,2.2999,0.9277,0.1768
5,200,85.0407,1.8894,1.0010,0.0724
7,50,20.5729,0.4224,6.1653,0.9750
7,100,43.2413,2.0630,10.0007,1.0963
7,150,69.4240,6.0489,13.4296,4.1152


In [3]:
def get_fig(table, k, stat, title):
    mers = table.filtered(f"k == {k}")
    nseqs = mers.columns["num_seqs"]
    stats = mers.columns[f"mean({stat})"]
    tr_data = go.Scatter(
        name="observed",
        x=nseqs,
        y=stats,
        error_y=dict(
            type="data",  # value of error bar given in data coordinates
            array=mers.columns[f"stdev({stat})"],
            visible=True,
        ),
    )
    tr_diag = go.Scatter(
        name="linear",
        x=(nseqs.min(), nseqs.max()),
        y=(stats.min(), stats.max()),
    )
    return go.Figure(
        data=[tr_data, tr_diag],
        layout=dict(
            title=title,
            xaxis=dict(title="number of sequences"),
            yaxis=dict(title="mean time (seconds)"),
            width=500,
            height=500,
        ),
    )

In [4]:
fig = get_fig(summary, 3, "s2r", "Converting sequences to records")
fig.show()

# Identifying the maximally divergent set

In [5]:
fig = get_fig(summary, 7, "maxd", "Identifying the most divergent sequences")
fig.show()